In [1]:
import torch

In [2]:
import importlib
componets = 'slr_network.SLRModel'.rsplit('.')
mod = importlib.import_module(componets[0])
mod = getattr(mod, componets[1])
mod

slr_network.SLRModel

In [4]:
import numpy as np

In [3]:
import yaml, utils

sparser = utils.get_parser()
p = sparser.parse_args([])
# p.config = "baseline_iter.yaml"
if p.config is not None:
    with open(p.config, 'r') as f:
        try:
            default_arg = yaml.load(f, Loader=yaml.FullLoader)
        except AttributeError:
            default_arg = yaml.load(f)
    key = vars(p).keys()
    for k in default_arg.keys():
        if k not in key:
            print('WRONG ARG: {}'.format(k))
            assert (k in key)
    sparser.set_defaults(**default_arg)
args = sparser.parse_args([])
with open(f"./configs/{args.dataset}.yaml", 'r') as f:
    args.dataset_info = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
arg=args
gloss_dict = np.load(arg.dataset_info['dict_path'], allow_pickle=True).item()


In [8]:
if arg.load_checkpoints or arg.load_weights:
    load_slowfast_pkl = False
else:
    load_slowfast_pkl = True

In [12]:
model = mod(
    num_classes=1296,
    c2d_type='slowfast101',
    conv_type=2,
    use_bn=1,
    share_classifier=1,
    weight_norm=True,
    gloss_dict=gloss_dict,
    loss_weights=arg.loss_weights,
    load_pkl=load_slowfast_pkl,
    slowfast_config=arg.slowfast_config,
    slowfast_args=arg.slowfast_args,
)

Use Bidirectional Add Fusion
res3_0_branch1_w: (512, 320, 1, 1, 1) => s3.pathway0_res0.branch1.weight: (512, 256, 1, 1, 1)
res5_0_branch2a_w: (512, 1280, 3, 1, 1) => s5.pathway0_res0.branch2.a.weight: (512, 1024, 3, 1, 1)
res4_0_branch2a_w: (256, 640, 3, 1, 1) => s4.pathway0_res0.branch2.a.weight: (256, 512, 3, 1, 1)
res2_0_branch2a_w: (64, 80, 1, 1, 1) => s2.pathway0_res0.branch2.a.weight: (64, 64, 1, 1, 1)
res3_0_branch2a_w: (128, 320, 1, 1, 1) => s3.pathway0_res0.branch2.a.weight: (128, 256, 1, 1, 1)
res2_0_branch1_w: (256, 80, 1, 1, 1) => s2.pathway0_res0.branch1.weight: (256, 64, 1, 1, 1)
res4_0_branch1_w: (1024, 640, 1, 1, 1) => s4.pathway0_res0.branch1.weight: (1024, 512, 1, 1, 1)
!! pred_b: can not be converted, got head.projection.bias
!! pred_w: can not be converted, got head.projection.weight
res5_0_branch1_w: (2048, 1280, 1, 1, 1) => s5.pathway0_res0.branch1.weight: (2048, 1024, 1, 1, 1)
Not loaded {'s2_fuse.bn2.bias', 's2_fuse.conv_f2s.weight', 's3_fuse.weight_f', 's4_fuse

In [10]:
arg.loss_weights, arg.slowfast_config, arg.slowfast_args

({'SeqCTC': 1.0, 'ConvCTC': 1.0, 'Dist': 25.0},
 'SLOWFAST_64x2_R101_50_50.yaml',
 {})

In [13]:
model

SLRModel(
  (conv2d): SlowFast(
    (loader): SlowFastLoader()
    (s1): VideoModelStem(
      (pathway0_stem): ResNetBasicStem(
        (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
      (pathway1_stem): ResNetBasicStem(
        (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        (bn): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
    )
    (s1_fuse): FuseBiAdd(
      (conv_f2s): Conv3d(8, 64, kernel_size=(5, 1, 1), stride=(4, 1, 1), padding=

In [18]:
model.conv1d

TemporalConv(
  (temporal_conv): Sequential(
    (0): Conv1d(2304, 1024, kernel_size=(5,), stride=(1,))
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,))
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): NormLinear()
)